In [ ]:
%pip install -q mlx mlx-lm mpi4py

In [ ]:
import mlx.core as mx
import sys, platform
print('MLX', mx.__version__)
print('Metal available', mx.metal.is_available())
print('Python', sys.version)
print('Platform', platform.platform())

In [ ]:
with open('distributed_infer.py','w') as f:
    f.write('''import mlx.core as mx
from mlx_lm import load, generate
import socket

world = mx.distributed.init()
rank = world.rank()
size = world.size()
mx.set_default_device(mx.gpu)

if rank == 0:
    print(f'Running on {size} processes')

model, tokenizer = load('mlx-community/Llama-3.2-1B-Instruct-4bit')
prompt = f'Hello from rank {rank}!'
result = generate(model, tokenizer, prompt, max_tokens=20)
print(f'[{rank}/{size} on {socket.gethostname()}] {result}')
''')

Run locally with two processes:
```bash
python -m mlx.launch --np 2 distributed_infer.py
```

Run on multiple hosts using a hostfile:
```bash
python -m mlx.launch --hostfile hosts.txt distributed_infer.py
```